<a href="https://colab.research.google.com/github/andresbadillo/LiquidacionAGPE/blob/master/LiquidacionAGPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En este primer paso se cargan las matrices

In [ ]:
# Paso 1: Importar librerías necesarias
import pandas as pd
from google.colab import files

# Paso 2: Crear una función para cargar y leer el archivo Excel
def cargar_matriz_consumo():
    print("Por favor, sube el archivo Excel con las matrices de consumo:")
    archivo_subido = files.upload()  # Sube el archivo desde tu máquina

    # Leer el archivo Excel en un DataFrame de pandas
    for nombre_archivo in archivo_subido.keys():
        try:
            df = pd.read_excel(nombre_archivo)
            print(f"Archivo '{nombre_archivo}' cargado con éxito.")
            return df
        except Exception as e:
            print(f"Error al leer el archivo '{nombre_archivo}': {e}")
            return None

# Paso 3: Llamar a la función para cargar el archivo
matriz_consumo = cargar_matriz_consumo()

# Paso 4: Mostrar una vista previa del archivo cargado, si es válido
if matriz_consumo is not None:
    print("Vista previa de los datos cargados:")
    print(matriz_consumo.head())
else:
    print("No se pudo cargar el archivo correctamente.")


Por favor, sube el archivo Excel con las matrices de consumo:


Saving Consumos_AGPE_Condominio_112024.xlsx to Consumos_AGPE_Condominio_112024.xlsx
Archivo 'Consumos_AGPE_Condominio_112024.xlsx' cargado con éxito.
Vista previa de los datos cargados:
    Año  Mes  Medidor    NIU                  Energía Sigla energía  Día  \
0  2024   11  2918909  49073  KiloWatt hour delivered          kWhD    1   
1  2024   11  2918909  49073  KiloWatt hour delivered          kWhD    2   
2  2024   11  2918909  49073  KiloWatt hour delivered          kWhD    3   
3  2024   11  2918909  49073  KiloWatt hour delivered          kWhD    4   
4  2024   11  2918909  49073  KiloWatt hour delivered          kWhD    5   

       Fecha    0    1  ...     14     15     16   17   18   19   20   21  \
0 2024-11-01  0.0  0.0  ...  5.790  3.094  0.032  0.0  0.0  0.0  0.0  0.0   
1 2024-11-02  0.0  0.0  ...  6.099  3.376  0.239  0.0  0.0  0.0  0.0  0.0   
2 2024-11-03  0.0  0.0  ...  5.303  1.970  1.016  0.0  0.0  0.0  0.0  0.0   
3 2024-11-04  0.0  0.0  ...  6.650  2.656  0.010 

En este paso eliminamos las filas con valores 'kVarhD' y 'kVarhR'

In [ ]:
# Paso 5: Tratamiento de los datos
def filtrar_datos(df):
    print("Eliminando filas con valores 'kVarhD' y 'kVarhR' en la columna 'Sigla energía'...")
    matriz_filtrada = df[~df['Sigla energía'].isin(['kVarhD', 'kVarhR'])]
    print("Filtrado completado. Vista previa de los datos filtrados:")
    print(matriz_filtrada.head())
    return matriz_filtrada

# Aplicar tratamiento si los datos fueron cargados correctamente
if matriz_consumo is not None:
    matriz_consumo_filtrada = filtrar_datos(matriz_consumo)

Eliminando filas con valores 'kVarhD' y 'kVarhR' en la columna 'Sigla energía'...
Filtrado completado. Vista previa de los datos filtrados:
    Año  Mes  Medidor    NIU                  Energía Sigla energía  Día  \
0  2024   11  2918909  49073  KiloWatt hour delivered          kWhD    1   
1  2024   11  2918909  49073  KiloWatt hour delivered          kWhD    2   
2  2024   11  2918909  49073  KiloWatt hour delivered          kWhD    3   
3  2024   11  2918909  49073  KiloWatt hour delivered          kWhD    4   
4  2024   11  2918909  49073  KiloWatt hour delivered          kWhD    5   

       Fecha    0    1  ...     14     15     16   17   18   19   20   21  \
0 2024-11-01  0.0  0.0  ...  5.790  3.094  0.032  0.0  0.0  0.0  0.0  0.0   
1 2024-11-02  0.0  0.0  ...  6.099  3.376  0.239  0.0  0.0  0.0  0.0  0.0   
2 2024-11-03  0.0  0.0  ...  5.303  1.970  1.016  0.0  0.0  0.0  0.0  0.0   
3 2024-11-04  0.0  0.0  ...  6.650  2.656  0.010  0.0  0.0  0.0  0.0  0.0   
4 2024-11-05  0.0 

En este paso generamos una tabla de la sumatoria de energía por NIU y energía

In [ ]:
# Paso 6: Crear tabla con la sumatoria mensual por NIU y Sigla energía
def crear_tabla_sumatoria_mensual(df):
    print("Creando tabla con la sumatoria mensual de los valores por NIU y Sigla energía...")
    columnas_horarias = [str(i) for i in range(24)]  # Columnas de 0 a 23
    df = df.copy()  # Crear una copia explícita del DataFrame para evitar advertencias
    df['Total Mensual'] = df[columnas_horarias].sum(axis=1)  # Sumar todas las columnas horarias por fila
    # Filtrar solo para 'kWhD' y 'kWhR'
    df = df[df['Sigla energía'].isin(['kWhD', 'kWhR'])]
    # Crear columnas separadas para kWhD y kWhR y ordenar
    tabla_sumatoria = df.pivot_table(
        index='NIU',
        columns='Sigla energía',
        values='Total Mensual',
        aggfunc='sum',
        fill_value=0
    ).reset_index()
    tabla_sumatoria = tabla_sumatoria.rename_axis(None, axis=1)  # Eliminar nombres de índices
    tabla_sumatoria = tabla_sumatoria.rename(columns={
        'kWhR': 'Energía importada',
        'kWhD': 'Energía exportada'
    })
    tabla_sumatoria = tabla_sumatoria[['NIU', 'Energía importada', 'Energía exportada']]  # Ordenar columnas
    print("Tabla de sumatoria mensual creada. Vista previa:")
    print(tabla_sumatoria.head())
    return tabla_sumatoria

# Generar la tabla de sumatoria mensual si los datos fueron filtrados correctamente
if 'matriz_consumo_filtrada' in locals():
    tabla_sumatoria_mensual = crear_tabla_sumatoria_mensual(matriz_consumo_filtrada)

Creando tabla con la sumatoria mensual de los valores por NIU y Sigla energía...
Tabla de sumatoria mensual creada. Vista previa:
   NIU  Energía importada  Energía exportada
0  227            665.810            356.740
1  234            402.912            533.454
2  505            954.010            436.990
3  515            432.314            435.984
4  520            862.638           1037.690


In [ ]:
# Paso 7: Añadir columnas calculadas
if 'tabla_sumatoria_mensual' in locals():
    print("Añadiendo columnas calculadas Exc1, Exc2 y Consumo...")
    tabla_sumatoria_mensual['Exc1'] = tabla_sumatoria_mensual.apply(
        lambda row: row['Energía exportada'] if row['Energía exportada'] <= row['Energía importada'] else row['Energía importada'], axis=1
    )
    tabla_sumatoria_mensual['Exc2'] = tabla_sumatoria_mensual.apply(
        lambda row: row['Energía exportada'] - row['Energía importada'] if row['Energía exportada'] > row['Energía importada'] else 0, axis=1
    )
    tabla_sumatoria_mensual['Consumo'] = tabla_sumatoria_mensual['Energía importada'] - tabla_sumatoria_mensual['Exc1']

    # Redondear las columnas a 2 decimales
    columnas_a_redondear = ['Energía importada', 'Energía exportada', 'Exc1', 'Exc2', 'Consumo']
    tabla_sumatoria_mensual[columnas_a_redondear] = tabla_sumatoria_mensual[columnas_a_redondear].round(2)

    print("Columnas añadidas y redondeadas a dos decimales. Vista previa:")
    print(tabla_sumatoria_mensual.head())

    # Exportar la tabla a Excel
    mes_anio = matriz_consumo['Fecha'].dt.strftime('%m_%Y').iloc[0]  # Obtener mes (mm) y año (AAAA) de la columna Fecha
    nombre_archivo = f"consumos_agpe_{mes_anio}.xlsx"
    tabla_sumatoria_mensual.to_excel(nombre_archivo, index=False)
    print(f"Tabla exportada a Excel con el nombre: {nombre_archivo}")


Añadiendo columnas calculadas Exc1, Exc2 y Consumo...
Columnas añadidas y redondeadas a dos decimales. Vista previa:
   NIU  Energía importada  Energía exportada    Exc1    Exc2  Consumo
0  227             665.81             356.74  356.74    0.00   309.07
1  234             402.91             533.45  402.91  130.54     0.00
2  505             954.01             436.99  436.99    0.00   517.02
3  515             432.31             435.98  432.31    3.67     0.00
4  520             862.64            1037.69  862.64  175.05     0.00
Tabla exportada a Excel con el nombre: consumos_agpe_11_2024.xlsx


En este paso generamos la tabla de los Excedentes 1 - Exc1

In [7]:
# Paso 8: Construir nueva tabla ajustada por consumo horario
if 'matriz_consumo_filtrada' in locals() and 'tabla_sumatoria_mensual' in locals():
    print("Construyendo nueva tabla ajustada por consumo horario basado en kWhD y kWhR...")

    def ajustar_consumos_por_niu(grupo, total_kWhR):
        acumulado = 0
        resultado = []

        for _, fila in grupo.iterrows():
            fila_ajustada = fila.copy()
            for hora in range(24):
                consumo_horario = fila[str(hora)]
                if acumulado + consumo_horario <= total_kWhR:
                    fila_ajustada[str(hora)] = consumo_horario
                    acumulado += consumo_horario
                else:
                    fila_ajustada[str(hora)] = total_kWhR - acumulado
                    acumulado = total_kWhR
                    break
            if acumulado >= total_kWhR:
                for hora_restante in range(hora + 1, 24):
                    fila_ajustada[str(hora_restante)] = 0
            resultado.append(fila_ajustada)

        return pd.DataFrame(resultado)

    nueva_tabla = []
    for niu, grupo in matriz_consumo_filtrada.groupby('NIU'):
        if 'Energía importada' in tabla_sumatoria_mensual.columns:
            total_kWhR = tabla_sumatoria_mensual.loc[tabla_sumatoria_mensual['NIU'] == niu, 'Energía importada'].iloc[0]
            grupo_filtrado = grupo[grupo['Sigla energía'] == 'kWhD']
            nueva_tabla.append(ajustar_consumos_por_niu(grupo_filtrado, total_kWhR))

    nueva_tabla = pd.concat(nueva_tabla, ignore_index=True)
    print("Nueva tabla ajustada construida. Vista previa:")
    print(nueva_tabla.head())

    # Exportar la nueva tabla a Excel
    nombre_archivo_ajustada = f"tabla_exc_1_{mes_anio}.xlsx"
    nueva_tabla.to_excel(nombre_archivo_ajustada, index=False)
    print(f"Nueva tabla ajustada exportada a Excel con el nombre: {nombre_archivo_ajustada}")

Construyendo nueva tabla ajustada por consumo horario basado en kWhD y kWhR...
Nueva tabla ajustada construida. Vista previa:
    Año  Mes   Medidor  NIU                  Energía Sigla energía  Día  \
0  2024   11  88606737  227  KiloWatt hour delivered          kWhD    1   
1  2024   11  88606737  227  KiloWatt hour delivered          kWhD    2   
2  2024   11  88606737  227  KiloWatt hour delivered          kWhD    3   
3  2024   11  88606737  227  KiloWatt hour delivered          kWhD    4   
4  2024   11  88606737  227  KiloWatt hour delivered          kWhD    5   

       Fecha    0    1  ...     14     15     16     17   18   19   20   21  \
0 2024-11-01  0.0  0.0  ...  1.640  0.734  0.206  0.000  0.0  0.0  0.0  0.0   
1 2024-11-02  0.0  0.0  ...  2.104  1.092  0.386  0.002  0.0  0.0  0.0  0.0   
2 2024-11-03  0.0  0.0  ...  0.798  0.476  0.080  0.000  0.0  0.0  0.0  0.0   
3 2024-11-04  0.0  0.0  ...  1.190  0.632  0.080  0.002  0.0  0.0  0.0  0.0   
4 2024-11-05  0.0  0.0  ... 

En este paso generamos la tabla de los Excedentes 2 - Exc2